In [14]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [15]:
!pip install tensorflow
!pip install tflite_support
!pip install flatbuffers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 23.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 30.5 MB/s eta 0:00:0000:01:00:01
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.1.1
    Uninstalling typing_extensions-4.1.1:
      Successfully uninstalled typing_extensions-4.1.1
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 23.1.4
    Uninstalling flatbuffers-23.1.4:
      Successfully uninstalled flatbuffers-23.1.4
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
  Attempting uninstall: h5py
    Found existing installation: h5py 3.7.0
    Uninstalling h5py-3.7.0:
      Successfully uninstalled h5py-3.7.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following 

In [16]:
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import json
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report,confusion_matrix


from tflite_support import metadata_schema_py_generated as _metadata_fb
from tflite_support import metadata as _metadata
import flatbuffers

In [17]:
IMAGES_PATH = "../input/citypersondata/leftImg8bit_trainvaltest/leftImg8bit"
ANNOTS_PATH = "../input/citypersondata/gtBboxCityPersonsWithDangerLevel/gtBboxCityPersonsWithDangerLevel"
BASE_OUTPUT = "output"

TRAIN_FILENAMES = "../input/citypersondata-annot/train.txt"
TEST_FILENAMES = "../input/citypersondata-annot/test.txt"
VAL_FILENAMES = "../input/citypersondata-annot/val.txt"

NUM_CLASSES = 3
OUTPUT_CLASSES = 4 #classes + background

INIT_LR = 1e-4
NUM_EPOCHS = 1
BATCH_SIZE = 16
BUFFER_SIZE = 40

IMG_H = 512#224#512 #384 #360
IMG_W = 512 #224 #512 #512 #480

In [18]:
def getClassFromJson(json_path):
    json_f = open(json_path)
    data = json.load(json_f)
    objects = data["objects"]
    highest_num = 0
    for o in objects:
        danger_level = None
        if "danger_level" in o:
            danger_level = int(o["danger_level"])
        else:
            danger_level = int(o["my_label"])
        
        
        if highest_num < danger_level:
            highest_num = danger_level
        if highest_num == 3:
            break
    return highest_num

In [19]:
def returnNamesLabels(filename):
    trainDataNames = []
    trainDataLabels = []
    f = open(filename, "r")
    for r in f:
        r = r.split(",") #image path, annotation path
        img_path = r[0]
        annot_path = r[1][:-1] #without \n
        class_num = getClassFromJson(annot_path)
        trainDataNames.append(img_path)
        trainDataLabels.append(class_num)
    f.close()
    return trainDataNames, trainDataLabels

In [20]:
trainDataNames, trainDataLabels = returnNamesLabels(TRAIN_FILENAMES)
valDataNames, valDataLabels = returnNamesLabels(VAL_FILENAMES)
testDataNames, testDataLabels = returnNamesLabels(TEST_FILENAMES)

allDataNames = trainDataNames + valDataNames + testDataNames
allDataLabels = trainDataLabels + valDataLabels + testDataLabels

In [21]:
X_train, X_test, y_train, y_test = train_test_split(allDataNames, allDataLabels, test_size=0.1438, random_state=42, shuffle=True, stratify=allDataLabels)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.122689, random_state=42, shuffle=True, stratify=y_train)

In [22]:
print(len(y_test))
print(len(y_train))
print(len(y_val))

STEPS_PER_EPOCH = len(y_train) // BATCH_SIZE

500
2610
365


In [23]:
def normalize(input_image, label):
    input_image = tf.cast(input_image, tf.float32) / 255.0
    return input_image, label

def read_image(image_path, h, w):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_png(image, channels=3)
    return tf.image.resize(image, [h, w])

def load_image(image_path, label, imageH, imageW):
    input_image = read_image(image_path, imageH, imageW)
    input_image, label = normalize(input_image, label)
    return input_image, label

In [24]:
class Augment(tf.keras.layers.Layer):
    def __init__(self, seed=42):
        super().__init__()
        # both use the same seed, so they'll make the same random changes.
        self.augment_inputs = tf.keras.layers.RandomFlip(mode="horizontal", seed=seed)

    def call(self, inputs, labels):
        inputs = self.augment_inputs(inputs)
        return inputs, labels

In [25]:
interpreter = tf.lite.Interpreter(model_path="/kaggle/input/mobilenet/mobilenet_06_12_9pm.tflite")
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
interpreter.allocate_tensors()

In [26]:
test_images = []
for img_path in X_test:
    img_arr = cv2.imread(img_path)[...,::-1]
    resized_arr = cv2.resize(img_arr, (IMG_H, IMG_W))
    test_images.append(resized_arr)

In [27]:
test_images = np.array(test_images, dtype="float32") / 255
test_images.reshape(-1, IMG_H, IMG_W, 1)
zzzzzzz = 0

In [28]:
#predictions = np.argmax(interpreter.predict(test_images, batch_size=BATCH_SIZE), axis=-1)
count_true = 0
count_false = 0
for i, new_img in enumerate(test_images): 
    interpreter.set_tensor(input_details[0]['index'], [new_img])
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    prediction = tf.nn.softmax(output_data[0])
    
    #print(prediction, np.max(prediction), np.argmax(prediction), y_test[i], y_test[i]==np.argmax(prediction))
    if np.max(prediction) > 0.70:
        print(np.max(prediction), np.argmax(prediction), y_test[i], y_test[i]==np.argmax(prediction))
        if y_test[i]==np.argmax(prediction):
            count_true += 1
        else:
            count_false += 1

2023-01-05 08:04:15.205110: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


0.79609066 1 1 True
0.7107028 1 2 False
0.72960496 1 1 True
0.81412387 1 1 True
0.77122426 1 1 True
0.75170773 1 1 True
0.7595215 1 1 True
0.71219057 1 1 True
0.9531153 3 3 True
0.7658435 1 1 True
0.7267275 1 1 True
0.73559546 1 1 True
0.745762 1 0 False
0.7185498 1 2 False
0.76082873 1 1 True
0.7344368 1 0 False
0.7367854 1 1 True
0.7336365 1 1 True
0.82420033 1 1 True
0.71700114 1 1 True
0.7235215 1 1 True
0.893096 3 2 False
0.73222274 1 1 True
0.7929529 1 1 True
0.75424594 1 1 True
0.73682064 1 2 False
0.8377285 1 1 True
0.7998611 1 2 False
0.7682537 1 1 True
0.75847846 1 1 True
0.8380248 1 1 True
0.7909627 1 1 True
0.74096704 1 2 False
0.75234073 1 1 True
0.7900589 1 2 False
0.7212747 1 1 True
0.94491494 3 3 True
0.8415304 1 2 False
0.73555994 1 1 True
0.770071 1 0 False
0.70557564 1 1 True
0.744934 1 0 False
0.7625135 1 0 False
0.7237138 1 1 True
0.81530166 3 3 True
0.7471345 1 0 False
0.8546185 3 3 True
0.7200756 1 1 True
0.7811946 1 1 True
0.765117 1 0 False
0.7030947 1 1 True
0

In [29]:
count_true/(count_true+count_false)

0.7642276422764228

In [30]:
(count_true+count_false)/len(test_images)

0.492